In [ ]:
# TEST 5: Feasibility Check
# ABSTRACT----------------------------------------------------------------------------------------------------------------
Due to the probabilistic and heuristic nature of quantum annealing, a low energy solution does not necessarily
mean it's feasible. This experiment introduces a feasibility checking test to make sure the candidate solution
satisfies every constraint in the model.
# METHOD------------------------------------------------------------------------------------------------------------------
After solving the model using the Exact CQM solver, the best candidate solution is selected.
Each constraint in the CQM is then evaluated independently by:

- Computing the numerical value of the left hand side.
- Comparing it against the constraint’s right-hand side according to its relation (<=, or >=).
- Recording any violation and its magnitude.
# ADDED CODE--------------------------------------------------------------------------------------------------------------
def check_feasibility(cqm, sample):
    """Return number of violated constraints and details."""
    violations = {}
    for label, constraint in cqm.constraints.items():
        lhs_val = constraint.lhs.energy(sample)
        rhs_val = constraint.rhs
        sense = constraint.sense

        if sense == "<=" and lhs_val > rhs_val:
            violations[label] = lhs_val - rhs_val
        elif sense == ">=" and lhs_val < rhs_val:
            violations[label] = rhs_val - lhs_val
        elif sense == "==" and lhs_val != rhs_val:
            violations[label] = abs(lhs_val - rhs_val)

    return violations
# Solve CQM and test feasibility
from dimod import ExactCQMSolver

solver = ExactCQMSolver()
sampleset = solver.sample_cqm(cqm)
# get best sample from SA
best_sample = sampleset.first.sample
violations = check_feasibility(cqm, best_sample)

if not violations:
    print("✅ Feasible! All constraints satisfied.")
else:
    print(f"❌ {len(violations)} violations:")
    for label, v in violations.items():
        print(f"  {label}: {v}")
# RESULTS-----------------------------------------------------------------------------------------------------------------
All constraints were satisfied exactly, confirming that the route assignments obeyed the customer
visitation rules, depot start/end requirements, and flow conservation.
# CONCLUSION--------------------------------------------------------------------------------------------------------------
This test was supposed to continue proving the robustness of the prototype as started in the previous test (exp 4).
However the previous test failed. Since this test has passed, this will help us isolate the problem and clearly
identify where the fault is at.

In [6]:
import dimod
from dimod import ConstrainedQuadraticModel, Binary
from dimod import quicksum

# Depot, Customers 1 and 2, Vehicles a and b
nodes = [0, 1, 2]          # 0 = depot
customers = [1, 2]
vehicles = ['a', 'b']

# Time dictionary (cost)
t = {
    (0,1): 3, (1,0): 3,
    (0,2): 5, (2,0): 5,
    (1,2): 4, (2,1): 4
}


# Create binary decision variables                   
# ---------------------------------
x = {}
for i in nodes:
    for j in nodes:
        if i != j: 
            for k in vehicles:
                name = f"x_{i}_{j}_{k}"
                x[i,j,k] = Binary(name)


# Building the CQM
# -----------------
cqm = ConstrainedQuadraticModel()

# 1) Objective: minimize total travel time
objective = quicksum(t[i,j] * x[i,j,k]
                     for i,j,k in x.keys())
cqm.set_objective(objective)

# Constraints
# -------------

# Each customer visited once
for v in customers:
    cqm.add_constraint(
        quicksum(x[i,v,k] for i in nodes if i != v for k in vehicles) == 1,
        label=f"Visit_{v}"
    )

# Start and end at Depot for each vehicle
for k in vehicles:
    # start from depot once
    cqm.add_constraint(
        quicksum(x[0,j,k] for j in customers) == 1,
        label=f"Start_{k}"
    )
    # return to depot once
    cqm.add_constraint(
        quicksum(x[i,0,k] for i in customers) == 1,
        label=f"End_{k}"
    )

# Flow conservation
for v in customers:
    for k in vehicles:
        cqm.add_constraint(
            quicksum(x[i,v,k] for i in nodes if i != v) -
            quicksum(x[v,j,k] for j in nodes if j != v) == 0,
            label=f"flow_{v}_{k}"
        )


# Converting to BQM 
bqm, invert = dimod.cqm_to_bqm(cqm)

# ----TEST----------------------------------------------------------------
def check_feasibility(cqm, sample):
    """Return number of violated constraints and details."""
    violations = {}
    for label, constraint in cqm.constraints.items():
        lhs_val = constraint.lhs.energy(sample)
        rhs_val = constraint.rhs
        sense = constraint.sense

        if sense == "<=" and lhs_val > rhs_val:
            violations[label] = lhs_val - rhs_val
        elif sense == ">=" and lhs_val < rhs_val:
            violations[label] = rhs_val - lhs_val
        elif sense == "==" and lhs_val != rhs_val:
            violations[label] = abs(lhs_val - rhs_val)

    return violations


# Solve CQM and test feasibility
from dimod import ExactCQMSolver

solver = ExactCQMSolver()
sampleset = solver.sample_cqm(cqm)


# get best sample from SA
best_sample = sampleset.first.sample

violations = check_feasibility(cqm, best_sample)

if not violations:
    print("✅ Feasible! All constraints satisfied.")
else:
    print(f"❌ {len(violations)} violations:")
    for label, v in violations.items():
        print(f"  {label}: {v}")
# -------------------------------------------------------------------------


✅ Feasible! All constraints satisfied.
